# Multi-Agents Orchestration Notebook

Ce notebook illustre un pipeline générique pour **finaliser un notebook cible** (quel qu’il soit), via la collaboration d’agents automatiques :
- **CoderAgent** : capable d’analyser et modifier le code ou le texte de cellules.
- **ReviewerAgent** : exécute le notebook, repère les problèmes potentiels et demande des corrections.
- **AdminAgent** : valide ou non la version finale.

**Utilisation** :
1. Sélectionner un notebook à finaliser (ou indiquer son chemin).
2. (Optionnel) Indiquer un objectif général (dans un champ texte).
3. Lancer la boucle d’orchestration et observer la conversation multi-agents.
4. Une fois le notebook approuvé par l’Admin, la conversation s’arrête automatiquement.

---


### Configuration de l'environnement

Cette cellule installe les dépendances nécessaires au bon fonctionnement du notebook :
- `papermill` pour l'exécution paramétrable de notebooks
- `nbformat` pour la manipulation des fichiers Jupyter
- `ipywidgets` pour les interactions utilisateur
- `semantic-kernel` pour la gestion des agents conversationnels

In [ ]:
# Cellule Code : Installation
%pip install papermill nbformat ipywidgets semantic-kernel ipyfilechooser  --quiet


### Import des bibliothèques

Importation des composants essentiels :
- Gestion de fichiers et JSON
- Logging avec personnalisation des couleurs
- Manipulation de notebooks via `nbformat`
- Intégration du Semantic Kernel pour les agents IA

In [ ]:
# Cellule Code : Imports et configuration
import os
import json
import hashlib
import logging
import nbformat
import papermill as pm
from datetime import datetime

# Pour la conversation multi-agents
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

# On importe la base pour nos stratégies
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.contents import ChatMessageContent, AuthorRole

### Journalisation avancée

Configuration d'un système de logging personnalisé :
- Niveaux de log colorés (DEBUG/INFO/WARNING/ERROR)
- Formatage horodaté des messages
- Sortie console avec gestion des verbosités

In [ ]:
class ColorFormatter(logging.Formatter):
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record):
        # Récupération d'informations optionnelles
        notebook_info = getattr(record, 'notebook_snapshot', '')
        agent_action = getattr(record, 'agent_action', '')
        msg = super().format(record)
        if notebook_info:
            msg += f"\n💻 NOTEBOOK CONTEXT: {notebook_info[:200]}..."
        if agent_action:
            msg += f"\n🤖 AGENT ACTION: {agent_action}"
        return f"{self.colors.get(record.levelname, '')}{msg}{self.reset}"


logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)

if not logger.handlers:
    ch = logging.StreamHandler()
    ch.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        "%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    ch.setFormatter(formatter)
    logger.addHandler(ch)

### Gestion d'état des notebooks

Implémentation de la classe `NotebookState` :
- Chargement/sauvegarde de notebooks
- Exécution via Papermill
- Mécanisme de cache et validation par hash MD5
- Gestion des états d'approbation

In [ ]:
import os
import json
import nbformat
import papermill as pm
import hashlib
import logging

class NotebookState:
    """
    Gère le statut et le contenu d'un notebook, 
    y compris la logique d'exécution papermill.
    Les 4 états sont: 
    - 'specified' (initial)
    - 'implemented'
    - 'tested'
    - 'validated'
    """

    def __init__(self, notebook_path: str):
        self.notebook_path = notebook_path
        self._cached_notebook = None
        # Par défaut, on commence en "specified"
        self._status = "specified"

        self._load_notebook()

    def _load_notebook(self):
        if not os.path.exists(self.notebook_path):
            raise FileNotFoundError(f"Notebook introuvable: {self.notebook_path}")

        with open(self.notebook_path, "r", encoding="utf-8") as f:
            self._cached_notebook = nbformat.read(f, as_version=4)

        logger.debug(f"[NotebookState] Notebook '{self.notebook_path}' chargé, "
                      f"nb de cellules={len(self._cached_notebook.cells)}.")

    def get_notebook_json(self) -> str:
        """
        Retourne le contenu actuel du notebook en mémoire au format JSON.
        """
        return json.dumps(self._cached_notebook, indent=2, ensure_ascii=False)
    
    def log_notebook_state(self, max_length: int = 4000):
       """Affiche le contenu JSON tronqué du notebook."""
       notebook_json = self.get_notebook_json()
       snippet = notebook_json[:max_length] + (">>> TRUNCATED <<<" if len(notebook_json) > max_length else "")
       logger.debug(f"[NotebookState] Current notebook state (truncated):\\n{snippet}")



    def get_status(self) -> str:
        """
        Retourne l'état courant du notebook: 'specified', 'implemented', 'tested', 'validated'.
        """
        return self._status

    def set_status(self, new_status: str):
        logger.info(f"[NotebookState] Passage de l'état {self._status} → {new_status}")
        self._status = new_status

    def save_notebook(self, path: str = ""):
        if not path:
            path = self.notebook_path

        try:
            with open(path, "w", encoding="utf-8") as f:
                nbformat.write(self._cached_notebook, f)
            logger.info(f"[NotebookState] Notebook sauvegardé sous {path}")
        except Exception as e:
            logger.error(f"[save_notebook] Erreur de sauvegarde: {e}")
            raise

        # relecture (optionnel)...

    def execute_notebook(self):
        """
        Exécute le notebook en place (via Papermill) et recharge le notebook,
        même en cas d'erreur d'exécution. Les erreurs sont logguées pour pouvoir
        être analysées par les agents.
        """
        logger.info(f"[NotebookState] Exécution Papermill sur {self.notebook_path}.")
        # Sauvegarde la version actuelle sur disque.
        self.save_notebook()  
        try:
            with open('execution.log', 'w') as log_file:
                pm.execute_notebook(
                    self.notebook_path,
                    self.notebook_path,
                    kernel_name="python3",
                    progress_bar=False,
                    log_output=True,
                    stdout_file=log_file,
                    stderr_file=log_file
                )
            # Optionnel : analyser le log pour détecter des messages d'erreur
            with open('execution.log', 'r', encoding='utf-8') as f:
                logs = f.read()
                if "Error" in logs or "Exception" in logs:
                    logger.error("[execute_notebook] Des erreurs ont été détectées lors de l'exécution.")
        except Exception as e:
            logger.error(f"[execute_notebook] Erreur lors de l'exécution: {e}")
            # Ne pas lever l'exception afin de permettre le rechargement du notebook
        finally:
            # Recharge le notebook, qu'il y ait eu ou non une erreur.
            self._load_notebook()
            logger.info("[NotebookState] Notebook mis à jour après exécution.")
    
    
    def update_cell(self, cell_index: int, new_source: str):
        """
        Met à jour immédiatement la source d'une cellule (puis save).
        """
        old_src = self._cached_notebook.cells[cell_index].source
        self._cached_notebook.cells[cell_index].source = new_source
        logger.info(f"[NotebookState] Mise à jour de la cellule {cell_index}")
        logger.debug(f"[NotebookState] Ancien contenu:\n{old_src}")
        logger.debug(f"[NotebookState] Nouveau contenu:\n{new_source}")

        self.save_notebook()
        logger.debug(f"[NotebookState] Cellule {cell_index} sauvegardée.")

    def find_cell_indices_by_content(self, content_pattern: str):
        indices = []
        for i, c in enumerate(self._cached_notebook.cells):
            if content_pattern in c.source:
                indices.append(i)
        return indices
    
    def is_approved(self) -> bool:
        """Retourne True si l'état du notebook est 'validated'."""
        return self._status == "validated"



### Upload d'un notebook personalisé (optionnel)

Si vous le souhaitez, vous pouvez utilisez le widget suivant pour uploader un notebook de votre choix à finaliser plutôt que le notebook par défaut.

Réexécutez les cellules suivantes le cas échéant.

In [ ]:
from IPython.display import display
import ipywidgets as widgets

# Widget d'upload
uploader = widgets.FileUpload(
    accept='.ipynb',
    multiple=False,
    description='Choisir notebook',
    disabled=False
)

display(uploader)

### Validation du système d'état

Procédure de test complète :
1. Copie d'un template de notebook
2. Injection dynamique d'une description de tâche
3. Exécution automatique du notebook
4. Vérification de l'intégrité du fichier

In [ ]:

import shutil
import logging


import random

POSSIBLE_TASKS = [
    "Créer un notebook Python permettant de requêter DBpedia (SPARQL) et d’afficher "
    "un graphique final avec rdflib ou SPARQLWrapper et plotly.\n\n"
    "**Résultat attendu** :\n"
    "1. Une requête complexe sur DBpedia (avec agrégats) correctement exécutée.\n"
    "2. Un graphique plotly **pertinent** reflétant les données issues de DBpedia.\n",
    "Créer un notebook Python qui charge le dataset Titanic depuis l'URL (https://raw.githubusercontent.com/datasciencedojo/datasets/refs/heads/master/titanic.csv) et effectue une analyse",
    "Construire un notebook scikit-learn sur le dataset IRIS disponible à cette url: (https://gist.githubusercontent.com/curran/a08a1080b88344b0c8a7/raw/0e7a9b0a5d22642a06d3d5b9bcbad9890c8ee534/iris.csv)"
]

chosen_task = random.choice(POSSIBLE_TASKS)


def apply_task_description(notebook_state: NotebookState, task_description: str) -> bool:
    for idx, cell in enumerate(notebook_state._cached_notebook.cells):
        if cell.cell_type == "markdown" and "## 0. Objectif du Notebook" in cell.source:
            new_source = cell.source.replace(
                "{{TASK_DESCRIPTION}}",
                f"<!-- TASK-ID: {hash(task_description)} -->\n{task_description}"
            )
            notebook_state.update_cell(idx, new_source)
            return True
    return False


SOURCE_NOTEBOOK = "Notebook-Template.ipynb"
DEST_NOTEBOOK = "Notebook-Generated.ipynb"


if uploader.value:
    fichier = uploader.value[0]
    DEST_NOTEBOOK = fichier.name
    
    # Sauvegarde du contenu
    with open(DEST_NOTEBOOK, 'wb') as f:
        f.write(fichier.content)
    
    logger.info(f"Notebook uploadé : {DEST_NOTEBOOK}")
else:
    # Comportement par défaut
    shutil.copy2(SOURCE_NOTEBOOK, DEST_NOTEBOOK)
    logger.info(f"Création depuis template : {DEST_NOTEBOOK}")



# 2) On charge le nouveau notebook dans NotebookState
notebook_state = NotebookState(DEST_NOTEBOOK)

# 3) On injecte la description DBpedia
changed = apply_task_description(notebook_state, chosen_task)

if changed:
    logger.info("Le placeholder DBpedia a bien été injecté.")

    # 4) Exécution du notebook en place (optionnel)
    logger.info("Exécution du notebook en place...")
    notebook_state.execute_notebook()  
    logger.info("Notebook ré-exécuté après injection de la tâche DBpedia.")

else:
    logger.warning("Aucun placeholder n'a été trouvé. Vérifiez que {{TASK_DESCRIPTION}} est présent.")


### Architecture de plugins

Système modulaire pour interagir avec le notebook :
- `BaseNotebookPlugin` : fonctionnalités de base
- `CoderPlugin` : édition des cellules
- `ReviewerPlugin` : validation par exécution
- `AdminPlugin` : approbation finale

In [ ]:

from semantic_kernel.functions import kernel_function

class BaseNotebookPlugin:
    """Classe de base pour manipuler NotebookState."""
    def __init__(self, state: NotebookState):
        self.state = state

    @kernel_function(
        name="get_notebook_content",
        description="Renvoie le notebook complet au format JSON (révision courante)."
    )
    def get_notebook_content(self) -> str:
        logger.info("get_notebook_content().")
        return self.state.get_notebook_json()



class CoderNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="update_cell_by_content",
        description="Modifie la première cellule qui contient 'content_pattern' (si unique)."
    )
    def update_cell_by_content(self, content_pattern: str, new_source: str) -> str:
        # Le coder modifie seulement si l'état n'est pas 'tested' ou 'validated'...
        logger.info("update_cell_by_content().")
        status = self.state.get_status()
        if status not in ["specified", "implemented"]:
            return f"Erreur: l'état du notebook est '{status}', je ne peux plus coder."
        
        indices = self.state.find_cell_indices_by_content(content_pattern)
        if len(indices) == 0:
            return f"Aucune cellule ne contient le motif '{content_pattern}'."
        if len(indices) > 1:
            return f"Plusieurs cellules contiennent '{content_pattern}', soyez plus précis."
        
        self.state.update_cell(indices[0], new_source)
        return f"Cellule avec motif '{content_pattern}' mise à jour."

    @kernel_function(
        name="finish_implementation",
        description="Marque le notebook comme 'implemented' quand le Coder a fini."
    )
    def finish_implementation(self) -> str:
        logger.info("finish_implementation().")
        status = self.state.get_status()
        if status == "specified":
            self.state.set_status("implemented")
            return "Le notebook passe en 'implemented'."
        elif status == "implemented":
            return "Le notebook est déjà en état 'implemented'."
        else:
            return f"Impossible de passer en 'implemented' depuis l'état '{status}'."



class ReviewerNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="validate_notebook",
        description="Exécute le notebook. Si approve=True => 'tested', sinon retour 'implemented'."
    )
    def validate_notebook(self, approve: bool = True) -> str:
        logger.info("validate_notebook({approve}).")
        status = self.state.get_status()
        if status != "implemented":
            return f"Le reviewer ne peut pas valider, l'état actuel est '{status}'."

        # On lit le notebook d'abord, si besoin
        # (Le reviewer peut d'abord exécuter get_notebook_content() manuellement)
        
        # Exécution
        self.state.execute_notebook()

        if approve:
            self.state.set_status("tested")
            return "Le reviewer valide => passage en 'tested'."
        else:
            # On redescend en 'implemented', pour que le Coder fasse des corrections
            self.state.set_status("specified")
            return "Le reviewer refuse => retour en 'specified'."



class AdminNotebookPlugin(BaseNotebookPlugin):
    @kernel_function(
        name="approve_notebook",
        description="Si admin_ok=True => 'validated', sinon retour 'implemented' (ou 'tested')."
    )
    def approve_notebook(self, admin_ok: bool = True) -> str:
        status = self.state.get_status()
        logger.info("approve_notebook({admin_ok}).")
        if status != "tested":
            return f"Impossible d'approuver, l'état est '{status}' au lieu de 'tested'."

        if admin_ok:
            self.state.set_status("validated")
            logger.info("[Admin] Validation finale - log notebook complet")
            return "Notebook validé => état 'validated'."
        else:
            # Par exemple, on retourne en 'implemented' si l'admin trouve qu'il faut recoder
            self.state.set_status("specified")
            return "L'admin refuse => retour en 'specified'."



### Stratégies d'orchestration

Politiques de contrôle du flux :
- `ApprovedBasedTermination` : arrêt sur approbation ou limite d'étapes
- `NotebookAwareSelection` : rotation cyclique entre agents
- Intégration des états du notebook dans les décisions

In [ ]:
from pydantic import PrivateAttr

class ApprovedBasedTerminationStrategy(TerminationStrategy):
    _state: NotebookState = PrivateAttr()
    _max_steps: int = PrivateAttr(default=10)  # Limite de sécurité

    def __init__(self, state: NotebookState, max_steps: int = 10):
        super().__init__()
        self._state = state
        self._max_steps = max_steps
        self._current_step = 0

    async def should_agent_terminate(self, agent, history):
        self._current_step += 1
        is_approved = self._state.is_approved()
        logger.debug(
            f"[TerminationStrategy] Step={self._current_step}/{self._max_steps}, "
            f"IsApproved={is_approved}"
        )

        if is_approved:
            logger.info("[TerminationStrategy] Notebook approuvé => on arrête.")
            return True

        if self._current_step >= self._max_steps:
            logger.warning(f"[TerminationStrategy] max_steps={self._max_steps} atteint => on arrête.")
            return True

        return False



# strategies.py

class NotebookAwareSelectionStrategy(SelectionStrategy):
    def __init__(self, state: NotebookState):
        super().__init__()
        self._state = state  # On stocke le NotebookState
        self._max_agents = 3  # éventuellement
        self._index = -1
    
    def reset(self) -> None:
        self._index = -1
    
    async def select_agent(self, agents, history):
        
        current_status = self._state.get_status()
        logger.debug(f"[SelectionStrategy] Nombre d'agents: {len(agents)}, Statut courant: {current_status}")
        
        if current_status == "validated":
            # Le notebook est déjà validé : plus personne ne parle
            logger.info("[SelectionStrategy] Le notebook est déjà VALIDÉ. Conversation terminée.")
            return None

        # On cherche par nom d’agent
        coder = next((a for a in agents if a.name == "CoderAgent"), None)
        reviewer = next((a for a in agents if a.name == "ReviewerAgent"), None)
        admin = next((a for a in agents if a.name == "AdminAgent"), None)

        # Sélection de l'agent en fonction de l’état
        if current_status == "specified" and coder:
            toReturn = coder
        elif current_status == "implemented" and reviewer:
            toReturn = reviewer
        elif current_status == "tested" and admin:
            toReturn = admin
        else:
            logger.warning(
                "[SelectionStrategy] Aucun agent approprié trouvé pour "
                f"l'état={current_status} => on arrête."
            )
            toReturn = None
        logger.info(f"Prochain Agent: {toReturn.name}")
        return toReturn


### Création des agents intelligents

Initialisation des trois agents spécialisés :
1. **CoderAgent** : 
   - Capacité d'édition via fonctions natives
   - Accès complet au contenu du notebook
   - Instructions détaillées pour les modifications

2. **ReviewerAgent** :
   - Exécution systématique après modifications
   - Analyse des résultats et détection d'erreurs
   - Feedback structuré pour le Coder

3. **AdminAgent** :
   - Décision finale d'approbation
   - Supervision du processus global
   - Verrouillage du workflow

In [ ]:
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions.kernel_arguments import KernelArguments

def create_kernel_for_agent(agent_id: str, plugin_instance) -> Kernel:
    k = Kernel()
    
    # 1) Ajout du service ChatCompletion
    k.add_service(OpenAIChatCompletion(service_id="default"))
    
    plugin_name=f"{agent_id}_plugin"
    # 2) Ajout du plugin
    k.add_plugin(plugin_instance, plugin_name=plugin_name)
    return k

# Instanciation des plugins
coder_plugin = CoderNotebookPlugin(notebook_state)
reviewer_plugin = ReviewerNotebookPlugin(notebook_state)
admin_plugin = AdminNotebookPlugin(notebook_state)

# Kernels
coder_kernel = create_kernel_for_agent("coder_kernel", coder_plugin)
reviewer_kernel = create_kernel_for_agent("reviewer_kernel", reviewer_plugin)
admin_kernel = create_kernel_for_agent("admin_kernel", admin_plugin)

# Configuration auto-invocation des fonctions
coder_settings = coder_kernel.get_prompt_execution_settings_from_service_id("default")
coder_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

reviewer_settings = reviewer_kernel.get_prompt_execution_settings_from_service_id("default")
reviewer_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

admin_settings = admin_kernel.get_prompt_execution_settings_from_service_id("default")
admin_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Agents
coder_agent = ChatCompletionAgent(
    service_id="coder_kernel",
    kernel=coder_kernel,
    name="CoderAgent",
    instructions=(
        "Vous êtes le **Coder**. Votre rôle :\n"
        "1. Appeler get_notebook_content() pour consulter le notebook à tout moment.\n"
        "2. Appliquer des modifications **au besoin** pour atteindre l'objectif global, corriger les cellules en erreur ou suite aux retours des autres agents dans la conversation.\n"
        "3. Identifier les cellules à modifier via des motifs uniques\n"
        "4. Exemple de marqueurs à utiliser :\n"
        "   - '# CELLULE-0'\n"
        "   - '# CELLULE-QUERY: SPARQL'\n"
        "5. Pour modifier : update_cell_by_content(motif, nouveau_contenu)\n"
        "6. Toujours conserver les marqueurs dans le code!\n"
        "7. Quand vous jugez que l’implémentation est terminée, appelez finish_implementation()\n"
        "8. Quand vous avez fini, résumez vos changements.\n"
        "Si le Reviewer ou l’Admin demandent des retouches, vous pouvez recommencer.\n"
    ),
    arguments=KernelArguments(settings=coder_settings)
)

reviewer_agent = ChatCompletionAgent(
    service_id="reviewer_kernel",
    kernel=reviewer_kernel,
    name="ReviewerAgent",
    instructions=(
        "Vous êtes le **Reviewer**. \n"
        "1) Avant de valider, appelez get_notebook_content() pour vérifier le code actuel.\n"
        "2) Notez le notebook avec validate_notebook(approve=...). S'il y a une erreur ou un point à améliorer pour l'objectif annoncé, passez 'approve' à False et sollicitez CoderAgent pour corriger.\n"
        "3) Argumentez votre décision (erreurs, graphes illisibles, etc.).\n"
    ),
    arguments=KernelArguments(settings=reviewer_settings)
)

admin_agent = ChatCompletionAgent(
    service_id="admin_kernel",
    kernel=admin_kernel,
    name="AdminAgent",
    instructions=(
        "Vous êtes l'**Admin**. \n"
        "1) Avant de prendre une décision, vous pouvez appeler get_notebook_content() pour lire les dernières modifications.\n"
        "2) Appelez approve_notebook(admin_ok=...). Si des cellules restent en erreur ou si les objectifs du notebook ne sont pas explicitement validés par les sorties de cellules, passez admin_ok a False \n"
        "3) Argumentez votre décision (erreurs, graphes illisibles, etc.).\n"
    ),
    arguments=KernelArguments(settings=admin_settings)
)


# GroupChat avec stratégies
termination_strategy = ApprovedBasedTerminationStrategy(notebook_state)
selection_strategy = NotebookAwareSelectionStrategy(notebook_state)

group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    selection_strategy=selection_strategy,
    termination_strategy=termination_strategy
)

logger.info("Agents créés, group_chat initialisé.")


### Boucle de conversation principale

Mécanisme d'exécution :
- Historique de conversation persistant
- Logging détaillé des appels de fonction
- Gestion asynchrone des interactions
- Conditions de sortie multiples
- Archivage des versions intermédiaires

In [ ]:
import asyncio
from semantic_kernel.contents.function_call_content import FunctionCallContent
from semantic_kernel.contents.function_result_content import FunctionResultContent

async def run_conversation():
    try:
        logger.info("Version initiale du  notebook")
        notebook_state.log_notebook_state()
        initial_content = notebook_state.get_notebook_json()
        group_chat.history.add_system_message(f"NOTEBOOK CONTENT:\\n{initial_content}")
        group_chat.history.add_user_message(
            "Bonjour, j'aimerais finaliser ce notebook qui contient des instructions et des cellules à compléter..."
        )
        
        iteration = 0
        async for message in group_chat.invoke():
            iteration += 1
            logger.debug(f"--- Iteration #{iteration} ---")
            
            # Log des appels de fonction
            for item in message.items:
                if isinstance(item, FunctionCallContent):
                    logger.debug(f"CALL: {item.name}({json.dumps(item.arguments, indent=2)[:100]}...)")  # Truncate

                if isinstance(item, FunctionResultContent):
                    logger.debug(f"RESULT: {str(item.result)[:200]}...")

            role = message.role.name
            content = message.content
            logger.info(f"{role.upper()} >>> {content}")

            if notebook_state.is_approved():
                logger.info("Notebook approuvé - arrêt de la conversation")
                break
            
        
        logger.info("Version finale sauvegardée dans le notebook")
        notebook_state.log_notebook_state(max_length=10000)

    except Exception as e:
        logger.error(f"Erreur inattendue: {str(e)}")
    finally:
        logger.info(f"Statut final - Approuvé: {notebook_state.is_approved()}")
        logger.info("Conversation terminée.")
        
        
await run_conversation()
